# ABC Guide for XMM-Newton -- RGS+EPIC Joint Spectral Fitting Part 1: Data Processing
<hr style="border: 2px solid #fadbac" />

- **Description:** XMM-Newton - Spectral Fitting Introduction
- **Level:** Advanced
- **Data:** XMM observation of Mkn 509 (obsid=0601390201)
- **Requirements:** Must be run using pySAS version 2.2.2 or higher.
- **Credit:** Jenna Cann (March 2025)
- **Support:** <a href="https://heasarc.gsfc.nasa.gov/docs/xmm/xmm_helpdesk.html">XMM Newton GOF Helpdesk</a>
- **Last verified to run:** 17 October 2025, for SAS v22.1 and pySAS v2.2.2

<hr style="border: 2px solid #fadbac" />

## 1. Introduction
This tutorial was created to guide XMM users through 
an example analysis of RGS and EPIC spectra. This is Part 1, where we will reduce and prepare the data for analysis in Part 2. 
#### Expected Outcome
The ability to fit RGS and EPIC spectra
#### SAS Tasks to be Used

- `rgsproc`[(Documentation for epproc)](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/rgsproc/index.html "rgsproc Documentation")
- `evselect`[(Documentation for evselect)](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/evselect/index.html)
- `tabgtigen`[(Documentation for tabgtigen)](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/tabgtigen/index.html)
- `gtibuild`[(Documentation for gtibuild)](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/gtibuild/index.html)

#### Useful Links

- [`pysas` Documentation](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/pysas/index.html "pysas Documentation")
- [`pysas` on GitHub](https://github.com/XMMGOF/pysas)
- [Common SAS Threads](https://www.cosmos.esa.int/web/xmm-newton/sas-threads "SAS Threads")
- [Users' Guide to the XMM-Newton Science Analysis System (SAS)](https://xmm-tools.cosmos.esa.int/external/xmm_user_support/documentation/sas_usg/USG/SASUSG.html "Users' Guide")
- [The XMM-Newton ABC Guide](https://heasarc.gsfc.nasa.gov/docs/xmm/abc/ "ABC Guide")
- [XMM Newton GOF Helpdesk](https://heasarc.gsfc.nasa.gov/docs/xmm/xmm_helpdesk.html "Helpdesk") - Link to form to contact the GOF Helpdesk.

#### Caveats
This tutorial uses an observation of Mkn 509 (obsid = '0601390201').

<div class="alert alert-block alert-warning">
    <b>Warning:</b> By default this notebook will place observation data files in your default <tt>data_dir</tt> directory. Make sure pySAS has been configured properly.
</div>

In [ ]:
# pySAS imports
import pysas
from pysas import MyTask

import xspec

# Useful imports
import os
import subprocess

# Imports for plotting
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
plt.style.use(astropy_mpl_style)

# To handle certain warnings
import warnings
warnings.filterwarnings("ignore")

## 2. Rerun basic processing

In [ ]:
obsid = '0601390201'

my_obs = pysas.ObsID(obsid)

We start by reprocessing the data. As we will be jointly fitting the MOS and RGS data, we will set run_emproc and run_rgsproc to 'True' and run_epproc to 'False'.

<div class="alert alert-block alert-info">
    <b>Note:</b> Running rgsproc on this particular obsid will take A LONG TIME, depending on your machine. Be prepared to wait.
</div>

In [ ]:
rgsproc_args = ["orders='1 2'",
                'bkgcorrect=no',
                'withmlambdacolumn=yes',
                'spectrumbinning=lambda']

my_obs.basic_setup(overwrite=False,rerun=False,
                   run_epproc=True,run_emproc=True,
                   run_rgsproc=True,rgsproc_args=rgsproc_args)

As we are processing both RGS and EPIC data, the above task will take several minutes. We will also follow the standard process for filtering EPIC data and extracting spectra from chapter 6 of the ABC guide.

In [ ]:
mos1 = my_obs.files['M1evt_list'][0]
mos2 = my_obs.files['M2evt_list'][0]
pn = my_obs.files['PNevt_list'][0]

print(mos1)
print(mos2)
print(pn)

In [ ]:
filtered_event_list_mos1 = 'mos1_filt1.fits'
filtered_event_list_mos2 = 'mos2_filt1.fits'
filtered_event_list_pn = 'pn_filt1.fits'

inargs = ['table={0}'.format(mos1), 
          'withfilteredset=yes', 
          "expression='(PATTERN <= 12)&&(PI in [200:12000])&&#XMMEA_EM'", 
          'filteredset={0}'.format(filtered_event_list_mos1), 
          'filtertype=expression', 
          'keepfilteroutput=yes', 
          'updateexposure=yes', 
          'filterexposure=yes']

MyTask('evselect', inargs).run()

inargs = ['table={0}'.format(mos2), 
          'withfilteredset=yes', 
          "expression='(PATTERN <= 12)&&(PI in [200:12000])&&#XMMEA_EM'", 
          'filteredset={0}'.format(filtered_event_list_mos2), 
          'filtertype=expression', 
          'keepfilteroutput=yes', 
          'updateexposure=yes', 
          'filterexposure=yes']

MyTask('evselect', inargs).run()

inargs = ['table={0}'.format(pn), 
          'withfilteredset=yes', 
          "expression='(PATTERN <= 4)&&(PI in [200:15000])&&#XMMEA_EP'", 
          'filteredset={0}'.format(filtered_event_list_pn), 
          'filtertype=expression', 
          'keepfilteroutput=yes', 
          'updateexposure=yes', 
          'filterexposure=yes']

MyTask('evselect', inargs).run()

Function for generating light curves.

In [ ]:
def plot_light_curve(event_list_file, filtered_event_list, light_curve_file, ccd):
    
    inargs = ['table={0}'.format(event_list_file), 
              'withfilteredset=yes', 
              "expression='(PATTERN == 0)&&(PI in [10000:12000])&&#XMMEA_{:s}'".format(ccd), 
              'filteredset={0}'.format(filtered_event_list), 
              'filtertype=expression', 
              'keepfilteroutput=yes', 
              'updateexposure=yes', 
              'filterexposure=yes']

    MyTask('evselect', inargs).run()

    inargs = ['table={0}'.format(filtered_event_list), 
              'withrateset=yes', 
              'rateset={0}'.format(light_curve_file), 
              'maketimecolumn=yes', 
              'timecolumn=TIME', 
              'timebinsize=100', 
              'makeratecolumn=yes']

    MyTask('evselect', inargs).run()

    ts = Table.read(light_curve_file,hdu=1)
    plt.plot(ts['TIME'],ts['RATE'])
    plt.xlabel('Time (s)')
    plt.ylabel('Count Rate (ct/s)')
    plt.show()

This particular observation is not contaminated with flaring, but as best practice we check the light curves to confirm that everything looks good. The light curves for a constant source like the one we are analyzing should remain fairly constant (with some noise of course). For your own sources you should of course check the light curves to check for contamination from flaring.

In [ ]:
light_curve_file_mos1='mos1_ltcrv.fits'
filtered_gtr10_mos1 = 'mos1_gtr10.fits'
plot_light_curve(filtered_event_list_mos1, filtered_gtr10_mos1, light_curve_file_mos1, 'EM')

light_curve_file_mos2='mos2_ltcrv.fits'
filtered_gtr10_mos2 = 'mos2_gtr10.fits'
plot_light_curve(filtered_event_list_mos2, filtered_gtr10_mos2, light_curve_file_mos2, 'EM')

light_curve_file_pn='pn_ltcrv.fits'
filtered_gtr10_pn = 'pn_gtr10.fits'
plot_light_curve(filtered_event_list_pn, filtered_gtr10_pn, light_curve_file_pn, 'EP')

In [ ]:
def make_spectrum(filtered_event_list, filtered_source, filtered_bkg, source_spectrum_file, bkg_spectrum_file, source_coords, bkg_coords, specchannelmax):

    inargs = {}
    inargs = {'table'           : filtered_event_list,
              'energycolumn'    : 'PI',
              'withfilteredset' : 'yes',
              'filteredset'     : filtered_source,
              'keepfilteroutput': 'yes',
              'filtertype'      : 'expression',
              'expression'      : "'((X,Y) in {:s})'".format(source_coords),
              'withspectrumset' : 'yes',
              'spectrumset'     : source_spectrum_file,
              'spectralbinsize' : '5',
              'withspecranges'  : 'yes',
              'specchannelmin'  : '0',
              'specchannelmax'  : '{:s}'.format(specchannelmax)}

    MyTask('evselect', inargs).run()

    inargs = {}
    inargs = {'table'           : filtered_event_list,
              'energycolumn'    : 'PI',
              'withfilteredset' : 'yes',
              'filteredset'     : filtered_bkg,
              'keepfilteroutput': 'yes',
              'filtertype'      : 'expression',
              'expression'      : "'((X,Y) in {:s})'".format(bkg_coords),
              'withspectrumset' : 'yes',
              'spectrumset'     : bkg_spectrum_file,
              'spectralbinsize' : '5',
              'withspecranges'  : 'yes',
              'specchannelmin'  : '0',
              'specchannelmax'  : '{:s}'.format(specchannelmax)}

    MyTask('evselect', inargs).run()
    
    return source_spectrum_file, bkg_spectrum_file

Let's use the `quick_eplot` command to identify our ideal source and background regions.

In [ ]:
my_obs.quick_eplot(mos1)

As there is limited area to extract a background spectrum in the pn image, we will focus on the spectra from the MOS1 and MOS2 images.

In [ ]:
filtered_source_mos1 = 'mos1_filt.fits'
filtered_bkg_mos1 = 'bkg_mos1_filt.fits'
source_spectrum_file_mos1 = 'mos1_pi.fits'
bkg_spectrum_file_mos1 = 'mos1_bkg_pi.fits'
source_coords_mos1 = 'CIRCLE(27623.01,26927.01,1033.12)'
bkg_coords_mos1 = 'CIRCLE(18052.94,35540.01,2697.07)'
specchannelmax_mos1 = '11999'

make_spectrum(filtered_event_list_mos1, filtered_source_mos1, 
              filtered_bkg_mos1, source_spectrum_file_mos1, 
              bkg_spectrum_file_mos1, source_coords_mos1, 
              bkg_coords_mos1, specchannelmax_mos1)

filtered_source_mos2 = 'mos2_filt.fits'
filtered_bkg_mos2 = 'bkg_mos2_filt.fits'
source_spectrum_file_mos2 = 'mos2_pi.fits'
bkg_spectrum_file_mos2 = 'mos2_bkg_pi.fits'
source_coords_mos2 = 'CIRCLE(27492.51,27144.51,1033.12)'
bkg_coords_mos2 = 'CIRCLE(14355.38,28101.50,2697.07)'
specchannelmax_mos2 = '11999'

make_spectrum(filtered_event_list_mos2, filtered_source_mos2, 
              filtered_bkg_mos2, source_spectrum_file_mos2, 
              bkg_spectrum_file_mos2, source_coords_mos2, 
              bkg_coords_mos2, specchannelmax_mos2)

In [ ]:
def make_rmf_arf(rmf,arf,source_spectrum_file,event_file):
    
    inargs = {'rmfset'      : rmf,
              'spectrumset' : source_spectrum_file}

    MyTask('rmfgen', inargs).run()

    inargs = {}
    inargs = {'arfset'         : arf,
              'spectrumset'    : source_spectrum_file,
              'withrmfset'     : 'yes',
              'rmfset'         : rmf,
              'withbadpixcorr' : 'yes',
              'badpixlocation' : event_file,
              'setbackscale'   : 'yes'}

    MyTask('arfgen', inargs).run()
    
    return rmf, arf

<div class="alert alert-block alert-info">
    <b>Note:</b> The next cell may take several minutes to run.
</div>

In [ ]:
mos1_rmf = 'mos1.rmf'
mos1_arf = 'mos1.arf'
mos2_rmf = 'mos2.rmf'
mos2_arf = 'mos2.arf'

make_rmf_arf(mos1_rmf, mos1_arf,source_spectrum_file_mos1,filtered_event_list_mos1)
make_rmf_arf(mos2_rmf, mos2_arf,source_spectrum_file_mos2,filtered_event_list_mos2)

In [ ]:
sou_mos1_grp = 'mos1_grp25.fits'
sou_mos2_grp = 'mos2_grp25.fits'

inargs = {}
inargs = {'spectrumset' : source_spectrum_file_mos1,
          'groupedset'  : sou_mos1_grp,
          'arfset'      : mos1_arf,
          'rmfset'      : mos1_rmf,
          'backgndset'  : bkg_spectrum_file_mos1,
          'mincounts'   : '25',
          'oversample'  : '3'}

MyTask('specgroup', inargs).run()

inargs = {}
inargs = {'spectrumset' : source_spectrum_file_mos2,
          'groupedset'  : sou_mos2_grp,
          'arfset'      : mos2_arf,
          'rmfset'      : mos2_rmf,
          'backgndset'  : bkg_spectrum_file_mos2,
          'mincounts'   : '25',
          'oversample'  : '3'}

MyTask('specgroup', inargs).run()

Now we have processed the relevant data and are ready to move forward with Part 2.